In [ ]:
#!pip install langchain-core langgraph>0.2.27

In [1]:
from langchain_ollama import ChatOllama

model = ChatOllama(
            model = "llama3.1",
            temperature = 0.8,
            num_predict = 256,
        )   

In [2]:
from langchain_core.messages import HumanMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Sohan")
    ]
)

AIMessage(content="Nice to meet you, Sohan! How's your day going so far? Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-11-23T02:58:32.233805Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 6484706625, 'load_duration': 4644087459, 'prompt_eval_count': 16, 'prompt_eval_duration': 206894000, 'eval_count': 32, 'eval_duration': 1631677000}, id='run-117484cb-af0a-4ce5-9b27-345a1f1d8b19-0', usage_metadata={'input_tokens': 16, 'output_tokens': 32, 'total_tokens': 48})

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are the quick assistant, reply only what is asked to you. dont give long responses. the response should be crisp but involves everything what question demands."
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [10]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    chain = prompt | model
    response = chain.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [11]:
config = {"configurable": {"thread_id": "conversation2"}}

In [12]:
query = "Hi! I'm Sohan."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hello Sohan, nice to meet you!


In [13]:
query = "how are you?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

I'm good, thanks for asking!


In [14]:
query = "who am i."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

You're Sohan.
